# Classifying Urban sounds using Deep Learning

## 4 Model Refinement 

### Load Preprocessed data 

In [1]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

#### Model refinement

In our inital attempt, we were able to achieve a Classification Accuracy score of: 

* Training data Accuracy:  92.3% 
* Testing data Accuracy:  87% 

We will now see if we can improve upon that score using a Convolutional Neural Network (CNN). 

#### Feature Extraction refinement 

In the prevous feature extraction stage, the MFCC vectors would vary in size for the different audio files (depending on the samples duration). 

However, CNNs require a fixed size for all inputs. To overcome this we will zero pad the output vectors to make them all the same size. 

In [2]:
import numpy as np
max_pad_len = 174

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

In [5]:
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'D:/Desktop_ANA_LeNovo/CLASES_UNIVERSIDAD/LABS/AUDIO/UrbanSound8K/audio/'

metadata = pd.read_csv('../UrbanSound Dataset sample/metadata/UrbanSound8K.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class_name"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files') 

Finished feature extraction from  8732  files


In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

### Convolutional Neural Network (CNN) model architecture 


We will modify our model to be a Convolutional Neural Network (CNN) again using Keras and a Tensorflow backend. 

Again we will use a `sequential` model, starting with a simple model architecture, consisting of four `Conv2D` convolution layers, with our final output layer being a `dense` layer. 

The convolution layers are designed for feature detection. It works by sliding a filter window over the input and performing a matrix multiplication and storing the result in a feature map. This operation is known as a convolution. 


The `filter` parameter specifies the number of nodes in each layer. Each layer will increase in size from 16, 32, 64 to 128, while the `kernel_size` parameter specifies the size of the kernel window which in this case is 2 resulting in a 2x2 filter matrix. 

The first layer will receive the input shape of (40, 174, 1) where 40 is the number of MFCC's 174 is the number of frames taking padding into account and the 1 signifying that the audio is mono. 

The activation function we will be using for our convolutional layers is `ReLU` which is the same as our previous model. We will use a smaller `Dropout` value of 20% on our convolutional layers. 

Each convolutional layer has an associated pooling layer of `MaxPooling2D` type with the final convolutional layer having a `GlobalAveragePooling2D` type. The pooling layer is do reduce the dimensionality of the model (by reducing the parameters and subsquent computation requirements) which serves to shorten the training time and reduce overfitting. The Max Pooling type takes the maximum size for each window and the Global Average Pooling type takes the average which is suitable for feeding into our `dense` output layer.  

Our output layer will have 10 nodes (num_labels) which matches the number of possible classifications. The activation is for our output layer is `softmax`. Softmax makes the output sum up to 1 so the output can be interpreted as probabilities. The model will then make its prediction based on which option has the highest probability.

In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 174
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax')) 

### Compiling the model 

For compiling our model, we will use the same three parameters as the previous model: 

In [8]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [9]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 39, 173, 16)       80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 86, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 86, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 85, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 42, 32)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 42, 32)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 41, 64)        

### Training 

Here we will train the model. As training a CNN can take a sigificant amount of time, we will start with a low number of epochs and a low batch size. If we can see from the output that the model is converging, we will increase both numbers.  

In [10]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

#num_epochs = 12
#num_batch_size = 128

num_epochs = 72
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 6985 samples, validate on 1747 samples
Epoch 1/72
6985/6985 [==============================] - 37s 5ms/step - loss: 3.3310 - accuracy: 0.2047 - val_loss: 2.0683 - val_accuracy: 0.2725

Epoch 00001: val_loss improved from inf to 2.06832, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 2/72
6985/6985 [==============================] - 36s 5ms/step - loss: 1.8387 - accuracy: 0.3578 - val_loss: 1.8689 - val_accuracy: 0.3898

Epoch 00002: val_loss improved from 2.06832 to 1.86888, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 3/72
6985/6985 [==============================] - 36s 5ms/step - loss: 1.5933 - accuracy: 0.4528 - val_loss: 1.6584 - val_accuracy: 0.4814

Epoch 00003: val_loss improved from 1.86888 to 1.65844, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 4/72
6985/6985 [==============================] - 36s 5ms/step - loss: 1.4504 - accuracy: 0.4963 - val_loss: 1.5376 - val_accuracy: 0.5209

Epoch 00004: val_loss improved fr

6985/6985 [==============================] - 31s 4ms/step - loss: 0.4732 - accuracy: 0.8361 - val_loss: 0.5165 - val_accuracy: 0.8432

Epoch 00034: val_loss improved from 0.57144 to 0.51654, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 35/72
6985/6985 [==============================] - 32s 5ms/step - loss: 0.4654 - accuracy: 0.8445 - val_loss: 0.5245 - val_accuracy: 0.8409

Epoch 00035: val_loss did not improve from 0.51654
Epoch 36/72
6985/6985 [==============================] - 33s 5ms/step - loss: 0.4780 - accuracy: 0.8366 - val_loss: 0.6075 - val_accuracy: 0.8105

Epoch 00036: val_loss did not improve from 0.51654
Epoch 37/72
6985/6985 [==============================] - 34s 5ms/step - loss: 0.4481 - accuracy: 0.8447 - val_loss: 0.4927 - val_accuracy: 0.8449

Epoch 00037: val_loss improved from 0.51654 to 0.49266, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 38/72
6985/6985 [==============================] - 35s 5ms/step - loss: 0.4398 - accuracy:

6985/6985 [==============================] - 26s 4ms/step - loss: 0.2491 - accuracy: 0.9114 - val_loss: 0.3326 - val_accuracy: 0.8878

Epoch 00071: val_loss did not improve from 0.33254
Epoch 72/72
6985/6985 [==============================] - 23s 3ms/step - loss: 0.2346 - accuracy: 0.9207 - val_loss: 0.3381 - val_accuracy: 0.8907

Epoch 00072: val_loss did not improve from 0.33254
Training completed in time:  0:42:45.282508


### Test the model 

Here we will review the accuracy of the model on both the training and test data sets. 

In [61]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9819613457408733
Testing Accuracy:  0.9192902116210514


The Training and Testing accuracy scores are both high and an increase on our initial model. Training accuracy has increased by ~6% and Testing accuracy has increased by ~4%. 

There is a marginal increase in the difference between the Training and Test scores (~6% compared to ~5% previously) though the difference remains low so the model has not suffered from overfitting. 

### Predictions  

Here we will modify our previous method for testing the models predictions on a specified audio .wav file. 

In [50]:
def print_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

### Validation 

#### Test with sample data 

As before we will verify the predictions using a subsection of the sample audio files we explored in the first notebook. We expect the bulk of these to be classified correctly. 

In [51]:
# Class: Air Conditioner

filename = '../UrbanSound Dataset sample/audio/100852-0-0-0.wav' 
print_prediction(filename) 

The predicted class is: air_conditioner 

air_conditioner 		 :  0.90663295984268188476562500000000
car_horn 		 :  0.00000379312382392527069896459579
children_playing 		 :  0.00372877437621355056762695312500
dog_bark 		 :  0.00003181818829034455120563507080
drilling 		 :  0.00387497572228312492370605468750
engine_idling 		 :  0.00299200275912880897521972656250
gun_shot 		 :  0.00765613839030265808105468750000
jackhammer 		 :  0.07329261302947998046875000000000
siren 		 :  0.00018024632299784570932388305664
street_music 		 :  0.00160682143177837133407592773438


In [52]:
# Class: Drilling

filename = '../UrbanSound Dataset sample/audio/103199-4-0-0.wav'
print_prediction(filename) 

The predicted class is: drilling 

air_conditioner 		 :  0.00070991273969411849975585937500
car_horn 		 :  0.00000001777174851724794280016795
children_playing 		 :  0.00001405069633619859814643859863
dog_bark 		 :  0.00000047111242906794359441846609
drilling 		 :  0.99598699808120727539062500000000
engine_idling 		 :  0.00000354658413925790227949619293
gun_shot 		 :  0.00000003223207656333215709310025
jackhammer 		 :  0.00052903906907886266708374023438
siren 		 :  0.00000098340262866258854046463966
street_music 		 :  0.00275487988255918025970458984375


In [53]:
# Class: Street music 

filename = '../UrbanSound Dataset sample/audio/101848-9-0-0.wav'
print_prediction(filename) 

The predicted class is: street_music 

air_conditioner 		 :  0.00011496015213197097182273864746
car_horn 		 :  0.00079288281267508864402770996094
children_playing 		 :  0.01791538484394550323486328125000
dog_bark 		 :  0.00257923710159957408905029296875
drilling 		 :  0.00007904539961600676178932189941
engine_idling 		 :  0.00006061193562345579266548156738
gun_shot 		 :  0.00000000007482268277181347571059
jackhammer 		 :  0.00000457825990451965481042861938
siren 		 :  0.00922307930886745452880859375000
street_music 		 :  0.96923023462295532226562500000000


In [64]:
# Class: Car Horn 

filename = '../UrbanSound Dataset sample/audio/100648-1-0-0.wav'
print_prediction(filename) 

The predicted class is: drilling 

air_conditioner 		 :  0.00059866637457162141799926757812
car_horn 		 :  0.26391193270683288574218750000000
children_playing 		 :  0.00126012135297060012817382812500
dog_bark 		 :  0.27843952178955078125000000000000
drilling 		 :  0.34817233681678771972656250000000
engine_idling 		 :  0.00339049054309725761413574218750
gun_shot 		 :  0.05176293104887008666992187500000
jackhammer 		 :  0.03859317675232887268066406250000
siren 		 :  0.01271206419914960861206054687500
street_music 		 :  0.00115874561015516519546508789062


#### Observations 

We can see that the model performs well. 

Interestingly, car horn was again incorrectly classifed but this time as drilling - though the per class confidence shows it was a close decision between car horn with 26% confidence and drilling at 34% confidence.  

### Other audio

Again we will further validate our model using a sample of various copyright free sounds that we not part of either our test or training data. 

In [55]:
filename = '../Evaluation audio/dog_bark_1.wav'
print_prediction(filename) 

The predicted class is: dog_bark 

air_conditioner 		 :  0.00053069164277985692024230957031
car_horn 		 :  0.01807974837720394134521484375000
children_playing 		 :  0.00958889070898294448852539062500
dog_bark 		 :  0.84292083978652954101562500000000
drilling 		 :  0.02251568622887134552001953125000
engine_idling 		 :  0.00286057707853615283966064453125
gun_shot 		 :  0.09233076870441436767578125000000
jackhammer 		 :  0.00147349410690367221832275390625
siren 		 :  0.00702858529984951019287109375000
street_music 		 :  0.00267084036022424697875976562500


In [66]:
filename = '../Evaluation audio/drilling_1.wav'

print_prediction(filename) 

The predicted class is: jackhammer 

air_conditioner 		 :  0.07861315459012985229492187500000
car_horn 		 :  0.00000012394852433317282702773809
children_playing 		 :  0.00000879450726642971858382225037
dog_bark 		 :  0.00000184070950126624666154384613
drilling 		 :  0.00003378492328920401632785797119
engine_idling 		 :  0.06372328102588653564453125000000
gun_shot 		 :  0.00000011736039340348725090734661
jackhammer 		 :  0.85761523246765136718750000000000
siren 		 :  0.00000361508728019543923437595367
street_music 		 :  0.00000013487000671830173814669251


In [65]:
filename = '../Evaluation audio/gun_shot_1.wav'

print_prediction(filename) 

The predicted class is: gun_shot 

air_conditioner 		 :  0.00000001711038777330031734891236
car_horn 		 :  0.00000002828730849557814508443698
children_playing 		 :  0.00001153892753791296854615211487
dog_bark 		 :  0.00006763751298421993851661682129
drilling 		 :  0.00002225582647952251136302947998
engine_idling 		 :  0.00000385214798370725475251674652
gun_shot 		 :  0.99988853931427001953125000000000
jackhammer 		 :  0.00000000060133342749679741245927
siren 		 :  0.00000603337139182258397340774536
street_music 		 :  0.00000002041979207945132657187060


#### Observations 

The performance of our final model is very good and has generalised well, seeming to predict well when tested against new audio data. 